In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def hessenberg(A):
    # init F
    F = np.zeros(A.shape) 
    for idx in range(1, A.shape[0]):
        F[idx, idx - 1] = 1.0
    
    # init Z
    Z = np.zeros(A.shape)
    Z[0, 0] = 1.0
    
    # recursive formula: Fik = (A * Zk)_i - sum_j=1^i-1 {Fjk * Zij} / Z_ii
    for k in range(1, A.shape[1] + 1):
        Azk = np.matmul(A, Z[:, k - 1])
        
        for i in range(0, k):
            temp = 0.0
            for j in range(0, i):
                temp += F[j, k - 1] * Z[i, j]
            F[i, k - 1] = (Azk[i] - temp) / Z[i, i]
        
        if k < A.shape[1]: # to get the last row of F, but here Z[:, k] would be out of range
            Z[:, k] = Azk[:]
            for t in range(0, k):
                Z[:, k] -= F[t, k - 1] * Z[:, t]
    
    return F, Z

def solve_qr(A, iterations=30):

    Ak = A
    Q_bar = np.eye(*Ak.shape)

    for k in range(iterations):
        Qk, Rk = np.linalg.qr(Ak)
        Ak = np.dot(Rk, Qk)
        Q_bar = Q_bar.dot(Qk)

    lam = np.diag(Ak)
    return lam, Q_bar

def solve_gen_eig_prob(A, B, eps=1e-6):
    """
    Solves the generalised eigenvalue problem of the form:
    Aw = \lambda*Bw
    
    Note: can be validated against `scipy.linalg.eig(A, b=B)`
    
    Ref: 
    'Eigenvalue and Generalized Eigenvalue Problems: Tutorial (2019)'
    Benyamin Ghojogh and Fakhri Karray and Mark Crowley
    arXiv 1903.11240

    """
    Lam_b, Phi_b = np.linalg.eig(B) # eig decomp of B alone
    Lam_b = np.eye(len(Lam_b))*Lam_b # convert to diagonal matrix of eig vals
    
    Lam_b_sq = replace_nan(Lam_b**0.5)+np.eye(len(Lam_b))*eps
    Phi_b_hat = np.dot(Phi_b, np.linalg.inv(Lam_b_sq))
    A_hat = np.dot(np.dot(Phi_b_hat.transpose(), A), Phi_b_hat)
    Lam_a, Phi_a = np.linalg.eig(A_hat)
    Lam_a = np.eye(len(Lam_a))*Lam_a
    
    Lam = Lam_a
    Phi = np.dot(Phi_b_hat, Phi_a)
    
    return np.diag(Lam), Phi

def solve_eig_qr(A, n_eig, lam_iterations=5):
    # !! note: eigenvectors can only be found reliably if A is symmetric
    Ak = A
    n_eig = min(n_eig, min(A.shape()))

    for k in range(lam_iterations):
        Qk, Rk = np.linalg.qr(Ak)
        Ak = np.dot(Rk, Qk)

    lam = np.diag(Ak) # get eigenvalues
    V = []
    for l in lam[:n_eig]: # now find `n_eig` eigenvectors
        A_null = (A - np.eye(A.shape()[0])*l).transpose()
        Q, R = np.linalg.qr(A_null) # compute null space of (A-lam*I) to get eigenvector
        V.append(Q[:, -1])
    return lam, np.array(V).transpose()

def power_iteration(A, iterations):
    """
    Iterative algo. to find the eigenvector of a matrix A corresponding to the largest
    eigenvalue.
    
    TODO: Establish some measure or heuristic of min number of iterations required
    """
    # choose random initial vector to reduce risk of choosing one orthogonal to 
    # target eigen vector
    b_k = np.array([urandom.random() for i in range(len(A))])

    for _ in range(iterations):
        b_k1 = np.dot(A, b_k)
        b_k1_norm = np.linalg.norm(b_k1)
        # re normalize the vector
        b_k = b_k1 / b_k1_norm

    return b_k1_norm, b_k

def max_eig(A, iterations, numeric_method='qr'):
    """
    Function to return the largest eigenvalue of a matrix and its corresponding eigenvector.
    
    A must be square but need not be symmetric. Tries to first use uLab `np.linalg.eig`
    that is better optimised but requires a symmetric matrix. Failing this, power iteration 
    algorithm is used.
    """
    try:
        lam, V = np.linalg.eig(A)
        v = V[:, np.argmax(lam)]
    except ValueError:
        if numeric_method == 'power_iteration':
            lam, v = power_iteration(A, iterations)
        else:
            if numeric_method != 'qr':
                print("Unknown `numeric_method` arg: defaulting to QR solver")
            lam, v = solve_eig_qr(A, 1, lam_iterations=iterations)
            lam = lam[0] # only need first eigen val (largest returned first)
            v = v[:, 0] # only first eig vector 
            
    return lam, v

def replace_nan(A, rep=0):
    return np.where(np.isfinite(A), A, rep)

In [4]:
class CCA():
    
    def __init__(self, stim_freqs, fs, Nh=2):
        self.Nh = Nh
        self.stim_freqs = stim_freqs
        self.fs = fs
        
    def compute_corr(self, X_test):            
        result = {}
        Cxx = np.dot(X_test, X_test.transpose()) # precompute data auto correlation matrix
        for f in self.stim_freqs:
            Y = harmonic_reference(f, self.fs, np.max(X_test.shape()), Nh=self.Nh, standardise_out=True)
            rho = self.cca_eig(X_test, Y, Cxx=Cxx) # canonical variable matrices. Xc = X^T.W_x
            result[f] = rho
        return result
    
    @staticmethod
    def cca_eig(X, Y, Cxx=None):
        if Cxx is None:
            Cxx = np.dot(X, X.transpose()) # auto correlation matrix
        Cyy = np.dot(Y, Y.transpose()) 
        Cxy = np.dot(X, Y.transpose()) # cross correlation matrix
        Cyx = np.dot(Y, X.transpose()) # same as Cxy.T

        M1 = np.dot(np.linalg.inv(Cxx), Cxy) # intermediate result
        M2 = np.dot(np.linalg.inv(Cyy), Cyx)

        lam, _ = max_eig(np.dot(M1, M2), 20)
        return np.sqrt(lam)

### CCA Example Problem
Testing finding canonical correlations using eigenvalues of covariance matrices. `X` is a data matrix and `Y` is a matrix of reference signals with 2 harmonics

In [5]:
from eeg_lib.synthetic import synth_X
from eeg_lib.cca import cca_reference

Ns = 150

X = synth_X(7, 1, Ns, noise_power=0.2, f_std=0.04)
Y = cca_reference([15], 250, Ns, Nh=2)

# X = X.T
# Y = Y.T

print(X.shape, Y.shape)

(1, 150) (4, 150)


In [6]:
from sklearn.cross_decomposition import CCA as CCA_sklearn

n = min(Y.T.shape[1], X.T.shape[1])
cca = CCA_sklearn(n_components=n)
cca.fit(X.T, Y.T)

X_c, Y_c = cca.transform(X.T, Y.T)
result = np.corrcoef(X_c.T, Y_c.T).diagonal(offset=n)
result

array([0.07173047])

In [7]:
Cxx = np.dot(X, X.transpose()) # auto correlation matrix
Cyy = np.dot(Y, Y.transpose()) 
Cxy = np.dot(X, Y.transpose()) # cross correlation matrix
Cyx = np.dot(Y, X.transpose()) # same as Cxy.T

M1 = np.dot(np.linalg.inv(Cxx), Cxy) # intermediate result
M2 = np.dot(np.linalg.inv(Cyy), Cyx)
M = np.dot(M1, M2)

lam, V = np.linalg.eig(M)
np.sqrt(lam)

array([0.07133839])

In [228]:
import ujson as json

data = {'X': X.tolist(), 'Y': Y.tolist()}
with open('xy.json', 'w') as jsonfile:
    json.dump(data, jsonfile)

In [216]:
lam, V = solve_qr(M, iterations=100)
np.sqrt(lam).round(5)

array([0.92299, 0.07742, 0.01606, 0.00484])

In [195]:
Cxx = X.dot(X.T)
Cxy = X.dot(Y.T)
Cyy = Y.dot(Y.T)
Cyx = Y.dot(X.T)

def block_diag(X, Y, reverse=False):
    if not reverse:
        X = np.concatenate([X, np.zeros_like(X)], axis=1)
        Y = np.concatenate([np.zeros_like(Y), Y], axis=1)
    else:
        X = np.concatenate([np.zeros_like(X), X], axis=1)
        Y = np.concatenate([Y, np.zeros_like(Y)], axis=1)
    return np.concatenate([X, Y], axis=0)

A = block_diag(Cxy, Cyx, reverse=True)
B = block_diag(Cxx, Cyy)

lam, Phi = solve_gen_eig_prob(A, B, eps=1e-12)

In [197]:
lam

array([ 0.96230964, -0.96230964,  0.6550056 , -0.6550056 ,  0.03841503,
       -0.03841503,  0.00107915, -0.00107915])

In [189]:
# args = [X, Y]
# Z = np.zeros((sum(arg.shape[0] for arg in args), sum(arg.shape[1] for arg in args)))
# origin = [0, 0]
# for arg in args:
#     x0 = origin[0]
#     y0 = origin[1]
#     Z[x0:x0+arg.shape[0], y0:y0+arg.shape[1]] = arg
#     origin[0] += arg.shape[0]
#     origin[1] += arg.shape[1]

# args = [X, Y]
# Z = np.zeros((sum(arg.shape[0] for arg in args), sum(arg.shape[1] for arg in args)))
# origin = Z.shape
# for arg in args:
#     x0 = origin[0]
#     y0 = origin[1]
#     Z[x0:x0-arg.shape[0], y0:y0-arg.shape[1]] = arg
#     origin[0] -= arg.shape[0]
#     origin[1] -= arg.shape[1]

In [79]:
A = np.array([2, 95, -38, 18, 5, 1, 47, -19, 8, 1, 2, 151, -69, 28, 4, -1, 218, -88, 34, 6, 0, -208, 84, -34, -5])
A = A.reshape((5,5))
A

array([[   2,   95,  -38,   18,    5],
       [   1,   47,  -19,    8,    1],
       [   2,  151,  -69,   28,    4],
       [  -1,  218,  -88,   34,    6],
       [   0, -208,   84,  -34,   -5]])

In [80]:
lam_ref, V_ref = np.linalg.eig(A)
print(f"Eigenvalues: \n{lam_ref}, \n\n Eigenvectors (rounded): \n{V_ref.round(2)}")

Eigenvalues: 
[ 25.57771123+0.j        -12.31982385+0.j         -3.21680685+1.4877582j
  -3.21680685-1.4877582j   2.17572632+0.j       ], 

 Eigenvectors (rounded): 
[[ 0.3 +0.j   -0.17+0.j    0.08+0.16j  0.08-0.16j  0.57+0.j  ]
 [ 0.16+0.j   -0.08+0.j   -0.01+0.01j -0.01-0.01j  0.  +0.j  ]
 [ 0.41+0.j   -0.51+0.j   -0.34+0.02j -0.34-0.02j -0.11+0.j  ]
 [ 0.58+0.j   -0.67+0.j   -0.83+0.j   -0.83-0.j   -0.43+0.j  ]
 [-0.62+0.j    0.51+0.j    0.38-0.11j  0.38+0.11j  0.69+0.j  ]]


In [81]:
lam, V = solve_qr(A, iterations=100)
lam

array([ 25.57771123, -12.31982385,  -1.81028516,  -4.62332854,
         2.17572632])

In [82]:
V.round(2)

array([[-0.3 , -0.54, -0.49, -0.42, -0.45],
       [-0.16, -0.34,  0.27,  0.74, -0.48],
       [-0.41,  0.49, -0.66,  0.39,  0.03],
       [-0.58,  0.44,  0.48, -0.33, -0.35],
       [ 0.62,  0.39, -0.15, -0.06, -0.66]])

In [83]:
A_f, _ = hessenberg(A)
lam_f, V_f = solve_qr(A_f, iterations=100)
lam_f

array([ 25.57771123, -12.31982385,  -3.09449956,  -3.33911414,
         2.17572632])

In [84]:
V_f.round(3)

array([[ 0.882, -0.205,  0.424,  0.001, -0.   ],
       [ 0.471,  0.413, -0.779, -0.003, -0.   ],
       [-0.015,  0.887,  0.461,  0.013,  0.   ],
       [ 0.001, -0.01 , -0.008,  0.981,  0.194],
       [ 0.   ,  0.002,  0.002, -0.194,  0.981]])

In [14]:
F, Z = hessenberg(A)
F.astype(int)

array([[     2,      1,    778,  35107, -15046],
       [     1,      1,    389,  17954,  -3009],
       [     0,      1,    -36,  -1527,    354],
       [     0,      0,      1,     42,     -9],
       [     0,      0,      0,      1,      0]])

In [85]:
X = np.array([[0.0, 0.2552531, 0.4935954, 0.6992362, 0.8585516, 0.9609866, 0.9997549, 0.972288, 0.8804055, 0.7301948],
       [0.0, 0.2651061, 0.5112409, 0.7207904, 0.8787591, 0.9738424, 0.9992362, 0.9531231, 0.838803, 0.6644569],
       [0.0, 0.2634635, 0.5083104, 0.7172394, 0.8754874, 0.9718722, 0.9995833, 0.9566626, 0.8461428, 0.6758333],
       [0.0, 0.2671577, 0.5148946, 0.7252015, 0.8827904, 0.9762053, 0.9986557, 0.9485094, 0.8294118, 0.6500207]])

# Y = cca_reference([7], 200, 10, Nh=2)

Y = np.array([[-2.171207, -1.338523, -0.5880827, 0.04396701, 0.5271821, 0.8382883, 0.9623005, 0.8932458, 0.6344502, 0.1983788],
       [1.305342, 1.170641, 0.9533603, 0.6639652, 0.3163952, -0.07260892, -0.4843098, -0.8988775, -1.296343, -1.657563],
       [0.2679634, 0.7796801, 1.073691, 1.094033, 0.8368343, 0.3510505, -0.2708505, -0.9104931, -1.446123, -1.775786],
       [1.850696, 1.432374, 0.8242437, 0.1420597, -0.4843266, -0.9356859, -1.126103, -1.019333, -0.6356997, -0.04822552]])

Cxx = X.dot(X.T)
Cxy = X.dot(Y.T)
Cyy = Y.dot(Y.T)
Cyx = Y.dot(X.T)

M1 = np.linalg.inv(Cxx).dot(Cxy)
M2 = np.linalg.inv(Cyy).dot(Cyx)

M = M1.dot(M2)

np.linalg.eig(M)

(array([0.02148729, 0.14447535, 1.00472133, 0.99984663]),
 array([[ 0.02902728,  0.02892596,  0.02260202,  0.02290183],
        [ 0.7939542 ,  0.79404442,  0.79942807,  0.79912809],
        [-0.53426395, -0.53399658, -0.51715765, -0.51825175],
        [-0.28871471, -0.28897127, -0.3048801 , -0.30378435]]))

In [86]:
M_f, _ = hessenberg(M)

In [87]:
lam, V = solve_qr(M_f, iterations=100)
lam

array([1.0048447 , 0.99972344, 0.14443503, 0.02152743])

In [88]:
V.round(3)

array([[ 1.   ,  0.031,  0.   , -0.   ],
       [-0.   , -0.   ,  0.031, -1.   ],
       [ 0.   ,  0.   , -1.   , -0.031],
       [ 0.031, -1.   , -0.   , -0.   ]])

In [89]:
V2 = []
for l in lam: # now find `n_eig` eigenvectors
    A_null = (M - np.eye(M.shape[0])*l).transpose()
    Q, R = np.linalg.qr(A_null) # compute null space of (A-lam*I) to get eigenvector
    V2.append(Q[:, -1])
    
V2 = np.array(V2).T
V2

array([[-0.02259485, -0.0228978 , -0.028926  , -0.02902725],
       [-0.79943385, -0.79915401, -0.79404439, -0.79395422],
       [ 0.51713865,  0.51812389,  0.53399668,  0.53426387],
       [ 0.30489771,  0.30393452,  0.28897118,  0.28871478]])

In [70]:
V2[:, np.argmax(lam)]

array([-0.02259485, -0.79943385,  0.51713865,  0.30489771])